# Master Climate-Commodity Dataset Analysis

This notebook demonstrates how to work with the master dataset that contains all commodity prices and climate variables in a single file. The master dataset provides a comprehensive view of climate conditions in multiple growing regions and prices for various commodities.

## Dataset Structure

The master dataset includes:
- Date information (Year, Month)
- Price columns for 7 commodities
- Region information for each commodity
- Climate variables for each commodity's growing region
- Temperature, precipitation, humidity and other climate metrics
- Derived climate signatures like anomalies and drought indices

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Set up plot parameters
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## 1. Load the Master Dataset

Let's load the master dataset and explore its structure.

In [ ]:
# Locate the master dataset
# Check in current directory first, then in organized/data_files
master_file = "MASTER_climate_commodity_data.csv"
if not os.path.exists(master_file):
    # Try the organized directory structure
    organized_path = "../data_files/MASTER_climate_commodity_data.csv"
    if os.path.exists(organized_path):
        master_file = organized_path
    # If in main directory, try the organized subdirectory
    elif os.path.exists("organized/data_files/MASTER_climate_commodity_data.csv"):
        master_file = "organized/data_files/MASTER_climate_commodity_data.csv"
    else:
        # If still not found, check for any path containing master dataset
        import glob
        master_files = glob.glob("**/MASTER_climate_commodity_data.csv", recursive=True)
        if master_files:
            master_file = master_files[0]

# Load the master dataset
if os.path.exists(master_file):
    df = pd.read_csv(master_file)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Display basic info
    print(f"Loaded master dataset from {master_file}")
    print(f"Dataset shape: {df.shape} (rows, columns)")
    print(f"Date range: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")
    
    # Identify column categories
    date_cols = ['Date', 'Year', 'Month']
    price_cols = [col for col in df.columns if col.endswith('_Price')]
    region_cols = [col for col in df.columns if col.endswith('_Region')]
    
    # Get the rest of the columns (climate variables)
    climate_cols = [col for col in df.columns 
                  if col not in date_cols + price_cols + region_cols]
    
    # Print column categories
    print(f"\nColumn categories:")
    print(f"  Date columns: {len(date_cols)} - {date_cols}")
    print(f"  Price columns: {len(price_cols)} - {price_cols}")
    print(f"  Region columns: {len(region_cols)} - {region_cols}")
    print(f"  Climate columns: {len(climate_cols)} (too many to list)")
    
    # Display first few rows
    df.head(3)
else:
    print(f"Error: Master dataset not found at {master_file}")
    print("Please run create_master_dataset.py to generate the master dataset.")
    df = None

## 2. Explore Commodity Prices

Let's start by looking at the price trends for all commodities.

In [ ]:
if df is not None and len(price_cols) > 0:
    # Create a figure for all price series
    plt.figure(figsize=(14, 8))
    
    # Plot actual prices first
    for price_col in price_cols:
        commodity = price_col.split('_')[0]
        plt.plot(df['Date'], df[price_col], label=f"{commodity} (actual)")
    
    plt.title('Commodity Prices Over Time', fontsize=14)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Now plot normalized prices (first value = 100)
    plt.figure(figsize=(14, 8))
    
    for price_col in price_cols:
        commodity = price_col.split('_')[0]
        # Normalize to first value = 100
        first_price = df[price_col].iloc[0]
        normalized_prices = df[price_col] / first_price * 100
        plt.plot(df['Date'], normalized_prices, label=commodity)
    
    plt.title('Normalized Commodity Prices (First Month = 100)', fontsize=14)
    plt.xlabel('Date')
    plt.ylabel('Price Index')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 3. Compare Regional Climate Patterns

Let's compare temperature and precipitation patterns across different commodity growing regions.

In [ ]:
if df is not None:
    # Define commodities and find their temperature and precipitation columns
    commodities = [col.split('_')[0] for col in price_cols]
    
    # Find temperature columns
    temp_cols = []
    for commodity in commodities:
        # Find columns containing 'temperature'
        for col in df.columns:
            if col.startswith(f"{commodity}_") and "temperature" in col:
                temp_cols.append(col)
                break
                
    # Find precipitation columns
    precip_cols = []
    for commodity in commodities:
        # Find columns containing 'precip'
        for col in df.columns:
            if col.startswith(f"{commodity}_") and "precip_m" in col:
                precip_cols.append(col)
                break
    
    # Plot temperature by region
    if temp_cols:
        plt.figure(figsize=(14, 8))
        
        for i, col in enumerate(temp_cols):
            commodity = col.split('_')[0]
            region_col = f"{commodity}_Region"
            region = df[region_col].iloc[0] if region_col in df.columns else commodity
            
            plt.plot(df['Date'], df[col], label=f"{region} ({commodity})")
        
        plt.title('Temperature by Region', fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # Plot precipitation by region
    if precip_cols:
        plt.figure(figsize=(14, 8))
        
        for i, col in enumerate(precip_cols):
            commodity = col.split('_')[0]
            region_col = f"{commodity}_Region"
            region = df[region_col].iloc[0] if region_col in df.columns else commodity
            
            plt.plot(df['Date'], df[col], label=f"{region} ({commodity})")
        
        plt.title('Precipitation by Region', fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Precipitation (m)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## 4. Heat Map of Climate-Price Correlations

Let's create a correlation heatmap between commodity prices and climate variables.

In [ ]:
if df is not None:
    # Get commodities
    commodities = [col.split('_')[0] for col in price_cols]
    
    # Create a matrix to store correlations
    # Rows: commodities, Columns: climate variables
    climate_vars = ['temperature_C', 'precip_m', 'drought_index']
    correlation_matrix = pd.DataFrame(index=commodities, columns=climate_vars)
    
    for commodity in commodities:
        price_col = f"{commodity}_Price"
        
        for var in climate_vars:
            # Find the column for this climate variable for this commodity
            var_col = None
            for col in df.columns:
                if col.startswith(f"{commodity}_") and var in col:
                    var_col = col
                    break
            
            # Calculate correlation if the column exists
            if var_col is not None and var_col in df.columns:
                correlation = df[var_col].corr(df[price_col])
                correlation_matrix.loc[commodity, var] = correlation
    
    # Fill any NaN values with 0 for visualization
    correlation_matrix = correlation_matrix.fillna(0)
    
    # Create a heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Correlation between Climate Variables and Commodity Prices', fontsize=14)
    plt.tight_layout()
    plt.show()

## 5. Regional Climate Impact on Coffee Prices

Let's analyze how climate variables from different regions correlate with coffee prices.

In [ ]:
if df is not None:
    # Focus on coffee prices
    target_commodity = 'Coffee'
    price_col = f"{target_commodity}_Price"
    
    if price_col in df.columns:
        # Create a matrix to store correlations
        # Rows: climate variables, Columns: regions
        climate_vars = ['temperature_C', 'precip_m', 'drought_index']
        regions_matrix = pd.DataFrame(index=climate_vars, columns=commodities)
        
        # For each variable, calculate correlation with coffee price for each region
        for var in climate_vars:
            for region_commodity in commodities:
                # Find the column for this climate variable for this region
                var_col = None
                for col in df.columns:
                    if col.startswith(f"{region_commodity}_") and var in col:
                        var_col = col
                        break
                
                # Calculate correlation if the column exists
                if var_col is not None and var_col in df.columns:
                    correlation = df[var_col].corr(df[price_col])
                    regions_matrix.loc[var, region_commodity] = correlation
        
        # Fill any NaN values with 0 for visualization
        regions_matrix = regions_matrix.fillna(0)
        
        # Create a heatmap
        plt.figure(figsize=(12, 8))
        sns.heatmap(regions_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
        plt.title(f'Regional Climate Correlations with {target_commodity} Prices', fontsize=14)
        plt.tight_layout()
        plt.show()

## 6. Seasonal Patterns in Price and Climate

Let's analyze how prices and climate variables change by month of the year.

In [ ]:
if df is not None:
    # Plot seasonal patterns for prices
    plt.figure(figsize=(14, 8))
    
    for price_col in price_cols:
        commodity = price_col.split('_')[0]
        
        # Calculate monthly averages
        monthly_avg = df.groupby('Month')[price_col].mean()
        
        # Normalize to percentage deviation from annual mean
        annual_mean = monthly_avg.mean()
        normalized_monthly = (monthly_avg / annual_mean - 1) * 100
        
        # Plot seasonal pattern
        plt.plot(monthly_avg.index, normalized_monthly.values, marker='o', linewidth=2, label=commodity)
    
    plt.title('Seasonal Price Patterns by Commodity', fontsize=14)
    plt.xlabel('Month')
    plt.ylabel('% Deviation from Annual Mean')
    plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Plot seasonal patterns for climate variables
    # Let's focus on temperature for coffee regions
    target_commodity = 'Coffee'
    temp_col = next((col for col in df.columns if col.startswith(f"{target_commodity}_") and "temperature" in col), None)
    
    if temp_col is not None:
        plt.figure(figsize=(14, 6))
        
        # Calculate monthly averages
        monthly_temp = df.groupby('Month')[temp_col].mean()
        
        # Plot seasonal pattern
        plt.plot(monthly_temp.index, monthly_temp.values, marker='o', linewidth=2, color='red')
        
        plt.title(f'Seasonal Temperature Pattern for {target_commodity} Region', fontsize=14)
        plt.xlabel('Month')
        plt.ylabel('Average Temperature (°C)')
        plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## 7. Climate Events and Price Movements

Let's examine how extreme climate events correspond to price movements.

In [ ]:
if df is not None:
    # Find drought index columns for all commodities
    drought_cols = []
    for commodity in commodities:
        # Find columns containing 'drought'
        for col in df.columns:
            if col.startswith(f"{commodity}_") and "drought" in col:
                drought_cols.append((commodity, col))
                break
    
    if drought_cols:
        # For each commodity, plot drought index and price
        for commodity, drought_col in drought_cols:
            price_col = f"{commodity}_Price"
            
            if price_col in df.columns:
                # Create a figure with two y-axes
                fig, ax1 = plt.subplots(figsize=(14, 6))
                
                # Plot drought index
                color = 'tab:red'
                ax1.set_xlabel('Date')
                ax1.set_ylabel('Drought Index', color=color)
                ax1.plot(df['Date'], df[drought_col], color=color)
                ax1.tick_params(axis='y', labelcolor=color)
                ax1.axhline(y=0, color='black', linestyle='--', alpha=0.3)
                
                # Create a second y-axis for price
                ax2 = ax1.twinx()
                color = 'tab:blue'
                ax2.set_ylabel(f'{commodity} Price', color=color)
                ax2.plot(df['Date'], df[price_col], color=color)
                ax2.tick_params(axis='y', labelcolor=color)
                
                # Identify drought periods (drought index < -0.5)
                drought_periods = df[df[drought_col] < -0.5]
                for i, row in drought_periods.iterrows():
                    ax1.axvspan(row['Date'], row['Date'] + pd.Timedelta(days=30), 
                               color='red', alpha=0.2)
                
                plt.title(f'Drought Index and Price for {commodity}', fontsize=14)
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()

## 8. Commodity Price Clustering

Let's use clustering to identify commodities with similar price behaviors.

In [ ]:
if df is not None and len(price_cols) >= 3:  # Need at least 3 commodities for meaningful clustering
    # Create a DataFrame with just the price data
    price_data = df[price_cols].copy()
    
    # Standardize the data
    scaler = StandardScaler()
    price_scaled = scaler.fit_transform(price_data)
    
    # Apply PCA to reduce dimensions to 2 for visualization
    pca = PCA(n_components=2)
    price_pca = pca.fit_transform(price_scaled)
    
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    clusters = kmeans.fit_predict(price_scaled)
    
    # Create a DataFrame with the PCA results and cluster assignments
    pca_df = pd.DataFrame(price_pca, columns=['Component 1', 'Component 2'])
    pca_df['Commodity'] = [col.split('_')[0] for col in price_cols]
    pca_df['Cluster'] = clusters
    
    # Plot the clusters
    plt.figure(figsize=(10, 8))
    
    for cluster_id in pca_df['Cluster'].unique():
        cluster_data = pca_df[pca_df['Cluster'] == cluster_id]
        plt.scatter(cluster_data['Component 1'], cluster_data['Component 2'], 
                   label=f'Cluster {cluster_id}', s=100)
    
    # Add commodity labels
    for i, row in pca_df.iterrows():
        plt.annotate(row['Commodity'], 
                    (row['Component 1'], row['Component 2']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=12, fontweight='bold')
    
    plt.title('Commodity Price Pattern Clustering', fontsize=14)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print cluster assignments
    print("Commodities by cluster:")
    for cluster_id in sorted(pca_df['Cluster'].unique()):
        commodities = pca_df[pca_df['Cluster'] == cluster_id]['Commodity'].tolist()
        print(f"Cluster {cluster_id}: {', '.join(commodities)}")

## 9. Time-Lagged Climate-Price Relationships

Let's analyze how climate conditions affect prices with a time lag.

In [ ]:
if df is not None:
    # Let's focus on coffee
    commodity = 'Coffee'
    price_col = f"{commodity}_Price"
    
    # Find temperature and precipitation columns for coffee
    temp_col = next((col for col in df.columns if col.startswith(f"{commodity}_") and "temperature" in col), None)
    precip_col = next((col for col in df.columns if col.startswith(f"{commodity}_") and "precip_m" in col), None)
    
    if price_col in df.columns and temp_col is not None and precip_col is not None:
        # Create a DataFrame with date, price, and climate variables
        lag_df = df[['Date', price_col, temp_col, precip_col]].copy()
        lag_df = lag_df.sort_values('Date')
        
        # Create lagged versions of climate variables (1-6 months)
        max_lag = 6
        
        # Calculate correlations for different lags
        temp_lag_corrs = []
        precip_lag_corrs = []
        
        for lag in range(max_lag + 1):
            # Create lagged columns
            temp_lag_col = f"{temp_col}_lag_{lag}"
            precip_lag_col = f"{precip_col}_lag_{lag}"
            
            lag_df[temp_lag_col] = lag_df[temp_col].shift(lag)
            lag_df[precip_lag_col] = lag_df[precip_col].shift(lag)
            
            # Calculate correlations
            temp_corr = lag_df[price_col].corr(lag_df[temp_lag_col])
            precip_corr = lag_df[price_col].corr(lag_df[precip_lag_col])
            
            temp_lag_corrs.append((lag, temp_corr))
            precip_lag_corrs.append((lag, precip_corr))
        
        # Plot the lag correlations
        plt.figure(figsize=(14, 6))
        
        # Temperature lag correlation
        plt.subplot(1, 2, 1)
        lags, corrs = zip(*temp_lag_corrs)
        plt.bar(lags, corrs, color='red', alpha=0.7)
        plt.title(f'Temperature → {commodity} Price Lag Correlation')
        plt.xlabel('Lag (months)')
        plt.ylabel('Correlation')
        plt.xticks(lags)
        plt.grid(True, alpha=0.3)
        
        # Precipitation lag correlation
        plt.subplot(1, 2, 2)
        lags, corrs = zip(*precip_lag_corrs)
        plt.bar(lags, corrs, color='blue', alpha=0.7)
        plt.title(f'Precipitation → {commodity} Price Lag Correlation')
        plt.xlabel('Lag (months)')
        plt.ylabel('Correlation')
        plt.xticks(lags)
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

## 10. Conclusions

The master dataset provides a comprehensive view of climate conditions and commodity prices, allowing for detailed analysis of their relationships. Key findings include:

1. **Commodity Price Trends**: Different commodities show distinct price patterns over time, with some commodities showing higher volatility than others.

2. **Regional Climate Variations**: Growing regions for different commodities have diverse climate profiles, with significant variations in temperature and precipitation patterns.

3. **Climate-Price Correlations**: Climate variables show varying levels of correlation with commodity prices, with some commodities being more sensitive to climate conditions than others.

4. **Seasonal Patterns**: Both prices and climate variables exhibit seasonal patterns, which can help explain price fluctuations throughout the year.

5. **Time-Lagged Effects**: Climate impacts on prices often show a time lag, reflecting the time it takes for climate conditions to affect crop yields and market prices.

This analysis demonstrates the value of combining climate and commodity data in a single comprehensive dataset, enabling a deeper understanding of the complex relationships between climate conditions and agricultural commodity markets.